In [1]:
from chromedriver_py import binary_path

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [3]:
from selenium.webdriver.common.by import By

In [4]:
import pandas as pd

In [51]:
driver = webdriver.Chrome(executable_path=binary_path)

<ipython-input-51-be9d09d8a619>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=binary_path)


In [8]:
def get_heads(html_part):
    lst = []
    head_found = False
    d_tbr = html_part.find_elements(By.XPATH,"//div[contains(@class, 'D(tbr)')]")
    for d in d_tbr:
        if not head_found:
            try:
                brk = d.find_elements(By.XPATH,".//span")
                for b in brk:
                    if b.text == 'Breakdown':
                        head_found = True
                    lst.append(b.text)
                    #print(b.text)
            except:
                print(ERROR)
    return lst


In [9]:
def get_span(reference_line, depth):
    item = []
    try:
        tbr = reference_line.find_element(By.XPATH,".//div[contains(@class, 'D(tbr)')]")
        span = tbr.find_elements(By.XPATH,".//span")
        i = 0
        for sp in span:
            #print("Título: ",i,sp.text," Nivel: ", depth)
            if i == 0:
                item.append(symbol)
                #print(item)
                item.append(depth)
            item.append(sp.text)
            i+=1
        financial_sheet.append(item)    
    except Exception as e:
        error = e.__class__.__name__
 
    

In [10]:
def no_button(error,reference_line,depth):
    e = error
    if e.__class__.__name__ == "NoSuchElementException":
        get_span(reference_line, depth)
    else:    
        error = e.__class__.__name__


In [11]:
def seek_buttons(html_part,depth = 0):
    tb = html_part.find_elements(By.XPATH,".//div[contains(@data-test, 'fin-row')]") 
    global fianacial_item
    if len(tb)>0:
        for ln in tb:
            try:
                button = ln.find_element(By.XPATH,".//button")
                name_class = button.get_attribute("class")
                get_span(ln, depth)
                button.click()
                seek_buttons(ln, depth = depth + 1)
            except Exception as e:
                no_button(e,ln,depth)


In [52]:
symbols = ['MSFT','AAPL']
df_balance = pd.DataFrame()

In [53]:
for symbol in symbols:
    financial_sheet = []
    url = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol
    driver.get(url)
    
    #heads
    tmp_heads = get_heads(driver)
    tmp_heads[0] = 'Title'
    heads = ['Company','Level',tmp_heads[0]]
    i = 0
    for th in tmp_heads[1:len(tmp_heads)]:
        heads.append("Year-" + str(i))
        i+=1
        
    #fill financial sheet
    seek_buttons(driver)
    
    #dataframe
    df_balance_company = pd.DataFrame(financial_sheet, columns = heads)
    if df_balance.empty :
        print("Symbol - empty: ",symbol)
        df_balance = df_balance_company.copy()
    else:
        print("Symbol - no empty: ",symbol)
        df_balance = df_balance.append(df_balance_company, sort = False)


Symbol - empty:  MSFT
Symbol - no empty:  AAPL


In [54]:
df_balance

,Company,Level,Title,Year-0,Year-1,Year-2,Year-3
0,MSFT,0,Total Assets,"333,779,000","301,311,000","286,556,000","258,848,000"
1,MSFT,1,Current Assets,"184,406,000","181,915,000","175,552,000","169,662,000"
2,MSFT,2,"Cash, Cash Equivalents & Short Term Investments","130,334,000","136,527,000","133,819,000","133,768,000"
3,MSFT,3,Cash And Cash Equivalents,"11,748,000","13,576,000","11,356,000","11,946,000"
4,MSFT,4,Cash,"7,272,000",None,None,None
...,...,...,...,...,...,...,...
55,AAPL,0,Tangible Book Value,"63,090,000","65,339,000","90,488,000","107,147,000"
56,AAPL,0,Total Debt,"124,719,000","112,436,000","108,047,000","114,483,000"
57,AAPL,0,Net Debt,"89,779,000","74,420,000","59,203,000","88,570,000"
58,AAPL,0,Share Issued,"16,426,786","16,976,763","17,772,944","19,019,944"
